In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\22498\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
symptoms = ["fever", "cough", "headache", "nausea", "fatigue", "diarrhea"]

# tokenize input text into words
input_text = "I have a fever and cough"
words = word_tokenize(input_text.lower())

# extract symptoms from input text
input_symptoms = []
for word in words:
    if word in symptoms:
        input_symptoms.append(word)

print(input_symptoms)

['fever', 'cough']


In [5]:
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')

def tokenize_words(sentences):
    """
    Tokenize the given list of sentences into words and return a list of the words.
    
    Args:
        sentences (list of str): The sentences to be tokenized.
    
    Returns:
        list of str: The list of words.
    """
    return nltk.word_tokenize(sentences)

def stem(word):
    """
    Reduce a word to its stem using the Porter stemming algorithm.
    
    Args:
        word (str): The word to be stemmed.
    
    Returns:
        str: The stemmed word.
    """
    stemmer = nltk.PorterStemmer()
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    """
    Convert a tokenized sentence to a bag of words vector.
    
    Args:
        tokenized_sentence (list of str): The tokenized sentence to be converted to a bag of words vector.
        words (list of str): The list of all words in the dataset.
    
    Returns:
        bag (ndarry): The bag of words vector.
    """
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, word in enumerate(words):
        if word in sentence_words:
            bag[idx] = 1.0
    return bag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\22498\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [135]:
import tensorflow as tf
import pickle
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.models import Sequential
import numpy as np
import random
import json
from nltk.tokenize import word_tokenize

def load_data():
    with open('intents.json') as file:
        data = json.load(file)
    return data

def create_tokenize():
    data = load_data()

    words = []
    labels = []
    docs_x = []
    docs_y = []
    ignore_words = ["?", ".", "!"]

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    labels = sorted(list(set(labels)))

    words = [stem(w.lower()) for w in words if w not in ignore_words]
    words = list(set(words))

    return words, labels, docs_x, docs_y

def create_training_data():
    words, labels, docs_x, docs_y = create_tokenize()
    training = []
    output = []
    out_empty = [0] * len(labels)

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)

    return training, output

def train_model():
    training, output = create_training_data()
    model = Sequential()
    model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(output[0]), activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    training, output = create_training_data()
    model.fit(training, output, epochs=200, batch_size=8, verbose=1)

    with open("model.pkl", "wb") as file:
        pickle.dump(model, file)

def predict_intent(sentence, words, classes):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stem(word.lower()) for word in sentence_words]
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1

    with open("model.pkl", "rb") as file:
        model = pickle.load(file)
    results = model.predict(np.array([bag]))[0]

    index = np.argmax(results)
    return classes[index]

def get_response(input_text, model, intents, words, classes, threshold=0.1):
    sentence_words = word_tokenize(input_text)
    sentence_words = [stem(word.lower()) for word in sentence_words]

    input_bag = bag_of_words(sentence_words, words)

    input_bag = np.array(input_bag).reshape(1, -1)
    results = model.predict([input_bag])[0]

    results = [[i, r] for i, r in enumerate(results) if r > threshold]

    results.sort(key=lambda x: x[1], reverse=True)

    response = "I'm sorry, I didn't understand that. Could you please try again?"

    if results:

        for result in results:
            tag = classes[result[0]]

            for intent in intents['intents']:
                if tag == intent["tag"]:
                    return random.choice(intent['responses'])
    return response

In [136]:
train_model()

THIS IS THE X: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
THIS IS THE NUMBER OF OUTPUT:  6
Epoch 1/200
6/6 [==============================] - 0s 1ms/step - loss: 1.7817 - accuracy: 0.2708
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6858 - accuracy: 0.4583
Epoch 3/200
6/6 [==============================] - 0s 998us/step - loss: 1.5795 - accuracy: 0.4375
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 1.4774 - accuracy: 0.4375
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3984 - accuracy: 0.4375
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3279 - accuracy: 0.4167
Epoch 7/200
6/6 [==============================] - 0s 1ms/step - loss: 1.2590 - accuracy: 0.4167
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.1938 - accuracy: 0.4583
Epoch 9/200
6/6 [==============================] - 0s 

In [141]:
words, classes, docs_x, docs_y = create_tokenize()
with open("model.pkl", "rb") as file:
    model = pickle.load(file)
with open('intents.json') as file:
    intents = json.load(file)
get_response("Who are you?", model, intents, words, classes, threshold=0.1)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-05 12:24:22         2050
metadata.json                                  2023-03-05 12:24:22           64
variables.h5                                   2023-03-05 12:24:22        85160
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...vars
1.Tokenize:  ['who', 'are', 'you', '?']
2.bag of words:  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1/1 [==============================] - 0s 34ms/step
This 

'My name is DocBot, a medical chatbot developed by Xufeng Bai. I am able to make predictions on diseases given the symptoms.'

In [96]:
create_model()

THIS IS THE X: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 1 0]
 [0 1 0 ... 0 1 0]
 [0 1 0 ... 0 0 1]]
THIS IS THE NUMBER OF OUTPUT:  6
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-04 16:53:02         2019
metadata.json                                  2023-03-04 16:53:02           64
variables.h5                                   2023-03-04 16:53:02        83200


c:\Users\22498\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
